# Exemple de notebook - filtres sql et récupération des données en pandas

L'objectif de ce notebook est de fournir des exemples pour pré-filtrer les données via sql avant de charger les données dans un DataFrame pandas. 


In [1]:
import pandas as pd

pd.set_option("display.max_columns", None)  # show all cols
pd.set_option("display.max_colwidth", None)  # show full width of showing cols
pd.set_option(
    "display.expand_frame_repr", False
)  # print cols side by side as it's supposed to be

In [2]:
# Nous commencons par importer les librairies nécessaires pour l'analyse des données.

import duckdb

con = duckdb.connect(database="./../../database/data.duckdb", read_only=True)

## Filtres
1. Filtrer sur les prélévements de 2024

In [3]:
query_2024 = """
SELECT * from edc_prelevements
WHERE dateprel >= '2024-01-01'
"""

prelevements_2024 = con.sql(query_2024)
prelevements_2024_df = prelevements_2024.df()
prelevements_2024_df.head(2)

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition
0,001,001000003,01007,AMBRONAY,None,None,None,00100139034,2024-01-23,11h27,Eau d'alimentation conforme aux limites de qualité et non conforme aux références de qualité.,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERIEU-EN-BUGEY,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERIEU-EN-BUGEY,C,C,N,C,2024
1,001,001000003,01007,AMBRONAY,None,None,None,00100139393,2024-02-13,12h49,Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés.,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERIEU-EN-BUGEY,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2024


2. Filtrer sur les prélévements non conformes en 2024


In [4]:
where_clause = """
"dateprel" >= '2024-01-01'
  AND (
    (
      "plvconformitebacterio" = 'N'
    )
    OR (
      "plvconformitechimique" = 'N'
    )
    OR (
      "plvconformitereferencebact" = 'N'
    )
    OR (
      "plvconformitereferencechim" = 'N'
    )
  )
"""
query_non_conforme = f"""
SELECT
  *
FROM "edc_prelevements"
WHERE
  {where_clause}
"""
prelevements_2024_non_conforme = con.sql(query_non_conforme)
prelevements_2024_non_conforme_df = prelevements_2024_non_conforme.df()
prelevements_2024_non_conforme_df.head(2)

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition
0,001,001000003,01007,AMBRONAY,None,None,None,00100142530,2024-10-18,13h18,Eau d'alimentation non-conforme aux exigences de qualité.,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERIEU-EN-BUGEY,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERIEU-EN-BUGEY,N,C,N,C,2024
1,001,001000244,01268,MURS-ET-GELIGNIEUX,001000242,TTP (UV) MURS-ET-GELIGNIEUX,100 %,00100141265,2024-07-10,11h59,Eau d'alimentation non-conforme aux exigences de qualité.,CC BUGEY SUD MURS-ET-GELIGNEUX,RÉGIE DES EAUX -CC BUGEY SUD,RÉGIE DES EAUX -CC BUGEY SUD,N,C,N,C,2024


## Selectionner des colonnes avant d'exécuter la requête

Selectionner les colonnes avant de charger les données permets une exécution plus rapide et limite l'usage de la mémoire.

In [5]:
query_preselected = f"""
SELECT
  "referenceprel",
  "dateprel",
  "nomcommuneprinc",
  "plvconformitebacterio"
FROM "edc_prelevements"
WHERE
  {where_clause}
"""
preselected = con.sql(query_preselected)
preselected_df = preselected.df()
preselected_df.head(2)

,referenceprel,dateprel,nomcommuneprinc,plvconformitebacterio
0,00100142530,2024-10-18,AMBRONAY,N
1,00100141265,2024-07-10,MURS-ET-GELIGNIEUX,N


## Jointure
Joindre edc_prelevements et edc_resultats sur referenceprel pour obtenir les résultats associés à chaque prélèvement :

In [6]:
query = f"""
SELECT
  "edc_prelevements"."referenceprel",
  "edc_prelevements"."dateprel",
  "edc_prelevements"."nomcommuneprinc",
  "edc_resultats"."libmajparametre",
  "edc_resultats"."insituana",
  "edc_resultats"."rqana",
  "edc_resultats"."cdunitereferencesiseeaux"
FROM (
  SELECT
    *
  FROM "edc_prelevements" 
  WHERE
    {where_clause}
) AS edc_prelevements
INNER JOIN "edc_resultats"
  ON "edc_prelevements"."referenceprel" = "edc_resultats"."referenceprel"
"""


joined = con.sql(query)
joined_df = joined.df()
joined_df

,referenceprel,dateprel,nomcommuneprinc,libmajparametre,insituana,rqana,cdunitereferencesiseeaux
0,00700198395,2024-03-14,AUBENAS,TÉTRACONAZOLE,L,"<0,005",µg/L
1,00700198395,2024-03-14,AUBENAS,TRICLOPYR,L,"<0,020",µg/L
2,00700198395,2024-03-14,AUBENAS,TEMPÉRATURE DE L'EAU,T,"13,5",°C
3,00700198395,2024-03-14,AUBENAS,TERBUMÉTON,L,"<0,005",µg/L
4,00700198395,2024-03-14,AUBENAS,TERBUMÉTON-DÉSETHYL,L,"<0,005",µg/L
...,...,...,...,...,...,...,...
6024762,97400138800,2024-07-22,TAMPON (LE),ENTÉROCOQUES /100ML-MS,L,<1,n/(100mL)
6024763,97400138800,2024-07-22,TAMPON (LE),TITRE ALCALIMÉTRIQUE,L,"<2,0",°f
6024764,97400138800,2024-07-22,TAMPON (LE),TEMPÉRATURE DE L'EAU (DOM),T,"18,0",°C
6024765,97400138800,2024-07-22,TAMPON (LE),TEMPÉRATURE DE MESURE DU PH,T,"18,0",°C


## Groupby et aggregats
Nombre total de prélèvements non conforme par commune en 2024

In [7]:
query = f"""
SELECT
  "nomcommuneprinc",
  COUNT("referenceprel") AS "nb_prelevements_non_conformes"
FROM (
  SELECT
    *
  FROM "edc_prelevements" 
  WHERE
    {where_clause}
) 
GROUP BY
  1
"""
grouped = con.sql(query)
grouped_df = grouped.df()
grouped_df.sort_values("nb_prelevements_non_conformes", ascending=False)

,nomcommuneprinc,nb_prelevements_non_conformes
13444,MARSEILLE,2016
8657,CAPESTERRE-BELLE-EAU,592
8508,AJACCIO,374
8667,SAINT-DENIS,344
10410,PREMESQUES,294
...,...,...
13778,BIGORNO,1
17,CURZAY-SUR-VONNE,1
11,JARD-SUR-MER,1
9,LANDERONDE,1


## Autres exemples :

In [8]:
# Exemple issu du notebook premier notebook d'exemple : exemple.ipynb
# Faisons une requête SQL en utilisant duckdb via la librarie python pour lister les substances qui ont été recherchées
# et les trier par ordre décroissant de leur nombre d'occurrences

con.sql("""
    SELECT libmajparametre, COUNT(*) as count
    FROM edc_resultats
    GROUP BY libmajparametre
    ORDER BY count DESC
""").show()

┌─────────────────────────────────────┬─────────┐
│           libmajparametre           │  count  │
│               varchar               │  int64  │
├─────────────────────────────────────┼─────────┤
│ PH                                  │ 3088944 │
│ CONDUCTIVITÉ À 25°C                 │ 2411835 │
│ TEMPÉRATURE DE L'EAU                │ 2379719 │
│ ENTÉROCOQUES /100ML-MS              │ 2339465 │
│ ESCHERICHIA COLI /100ML - MF        │ 2339452 │
│ BACTÉRIES COLIFORMES /100ML-MS      │ 2338332 │
│ TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU       │ 2323986 │
│ CHLORE LIBRE                        │ 2304946 │
│ BACT. AÉR. REVIVIFIABLES À 36°-44H  │ 2299438 │
│ BACT. AÉR. REVIVIFIABLES À 22°-68H  │ 2289233 │
│           ·                         │       · │
│           ·                         │       · │
│           ·                         │       · │
│ PHTALATE DE DIHEPTYLE               │       1 │
│ COLIFORMES ET ECOLI EN 24H          │       1 │
│ CHLOROTHALONIL MÉTABOLITE SYN507900 │       1 │


In [10]:
# Exemple issu du notebook premier notebook d'exemple : exemple.ipynb

# Enfin, terminons par lister les prélèvements effectués dans une commune donnée

nomcommune = "TOULOUSE"

con.sql(f"""
    SELECT *
    FROM edc_prelevements
    WHERE nomcommuneprinc = '{nomcommune}'
""").show()

┌─────────┬───────────┬───────────────────┬─────────────────┬───────────────┬───────────────────────────┬───────────────┬───────────────┬────────────┬───────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────┬──────────┬────────────────────┬───────────────────────┬───────────────────────┬────────────────────────────┬────────────────────────────┬──────────────┐
│ cddept  │ cdreseau  │ inseecommuneprinc │ nomcommuneprinc │ cdreseauamont │      nomreseauamont       │ pourcentdebit │ referenceprel │  dateprel  │ heureprel │                                                                                                                                                       